In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Data Scraping for NFL Data

In [3]:
# function that scrapes data from pro-football-reference and returns it as a dataframe
def scrape(url, start_idx):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')
    headers = []
    for th in table.find('thead').find_all('th'):
        headers.append(th.text.strip())
    headers = headers[start_idx:] # start_idx variable is needed because different tables have different # of columns
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        row = []
        for td in tr.find_all('td'):
            row.append(td.text.strip())
        rows.append(row)
    df = pd.DataFrame(rows, columns=headers)
    df['OvRank'] = range(1, len(df)+1)
    return df

In [4]:
# loop that uses the scrape function above to scrape data from all the desired years
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
dfs = []
for year in years:
    url = f'https://www.pro-football-reference.com/years/{year}/fantasy.htm'
    df_year = scrape(url, 11)
    df_year['Year'] = year
    dfs.append(df_year)
df_nfl = pd.concat(dfs, ignore_index=True)

# Data Cleaning and Database Setup

In [5]:
# checking column names
columns = df_nfl.columns.tolist()
columns

['Player',
 'Tm',
 'FantPos',
 'Age',
 'G',
 'GS',
 'Cmp',
 'Att',
 'Yds',
 'TD',
 'Int',
 'Att',
 'Yds',
 'Y/A',
 'TD',
 'Tgt',
 'Rec',
 'Yds',
 'Y/R',
 'TD',
 'Fmb',
 'FL',
 'TD',
 '2PM',
 '2PP',
 'FantPt',
 'PPR',
 'DKPt',
 'FDPt',
 'VBD',
 'PosRank',
 'OvRank',
 'Year']

In [6]:
# adjusting column names because there were columns with the same name
columns[7] = 'Pass_Att'
columns[8] = 'Pass_Yds'
columns[9] = 'Pass_TD'
columns[11] = 'Rush_Att'
columns[12] = 'Rush_Yds'
columns[14] = 'Rush_TD'
columns[17] = 'Rec_Yds'
columns[19] = 'Rec_TD'
columns[22] = 'Tot_TD'

In [7]:
# column names look good now
columns

['Player',
 'Tm',
 'FantPos',
 'Age',
 'G',
 'GS',
 'Cmp',
 'Pass_Att',
 'Pass_Yds',
 'Pass_TD',
 'Int',
 'Rush_Att',
 'Rush_Yds',
 'Y/A',
 'Rush_TD',
 'Tgt',
 'Rec',
 'Rec_Yds',
 'Y/R',
 'Rec_TD',
 'Fmb',
 'FL',
 'Tot_TD',
 '2PM',
 '2PP',
 'FantPt',
 'PPR',
 'DKPt',
 'FDPt',
 'VBD',
 'PosRank',
 'OvRank',
 'Year']

In [8]:
# setting column names in the dataframe to the column named I just modified
df_nfl.columns = columns
df_nfl.head()

,Player,Tm,FantPos,Age,G,GS,Cmp,Pass_Att,Pass_Yds,Pass_TD,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,Year
0,Antonio Brown*+,PIT,WR,27,16,16,0,0,0,0,...,2,,252,388.2,393.2,320.2,125,1,1,2015
1,Devonta Freeman*,ATL,RB,23,15,13,0,0,0,0,...,,,243,316.4,324.4,279.9,120,1,2,2015
2,Julio Jones*+,ATL,WR,26,16,16,0,0,0,0,...,,,239,375.1,379.1,307.1,111,2,3,2015
3,Adrian Peterson*+,MIN,RB,30,16,16,0,0,0,0,...,,,231,260.7,269.7,245.7,107,2,4,2015
4,Cam Newton*+,CAR,QB,26,16,16,296,495,3837,35,...,,,389,389.1,409.1,399.1,105,1,5,2015


In [9]:
# checking for missing data
df_nfl.isnull().sum()

Player      184
Tm          184
FantPos     184
Age         184
G           184
GS          184
Cmp         184
Pass_Att    184
Pass_Yds    184
Pass_TD     184
Int         184
Rush_Att    184
Rush_Yds    184
Y/A         184
Rush_TD     184
Tgt         184
Rec         184
Rec_Yds     184
Y/R         184
Rec_TD      184
Fmb         184
FL          184
Tot_TD      184
2PM         184
2PP         184
FantPt      184
PPR         184
DKPt        184
FDPt        184
VBD         184
PosRank     184
OvRank        0
Year          0
dtype: int64

In [10]:
# dropping all null rows because they were placeholder rows that contain no data in the original dataset
df_nfl.dropna(inplace=True)
df_nfl.isnull().sum()

Player      0
Tm          0
FantPos     0
Age         0
G           0
GS          0
Cmp         0
Pass_Att    0
Pass_Yds    0
Pass_TD     0
Int         0
Rush_Att    0
Rush_Yds    0
Y/A         0
Rush_TD     0
Tgt         0
Rec         0
Rec_Yds     0
Y/R         0
Rec_TD      0
Fmb         0
FL          0
Tot_TD      0
2PM         0
2PP         0
FantPt      0
PPR         0
DKPt        0
FDPt        0
VBD         0
PosRank     0
OvRank      0
Year        0
dtype: int64

In [11]:
# some of the player names have special characters after players name which denote Pro Bowl and All-Pro selections
df_nfl['Player'].head()

0      Antonio Brown*+
1     Devonta Freeman*
2        Julio Jones*+
3    Adrian Peterson*+
4         Cam Newton*+
Name: Player, dtype: object

In [12]:
# getting rid of these characters which will later allow for joins between years on player name
df_nfl['Player'] = df_nfl['Player'].str.replace(r'[\*\+]', '', regex=True)
df_nfl['Player'].head()

0      Antonio Brown
1    Devonta Freeman
2        Julio Jones
3    Adrian Peterson
4         Cam Newton
Name: Player, dtype: object

In [13]:
# checking the datatypes of the columns in the dataframe
df_nfl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5655 entries, 0 to 5838
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Player    5655 non-null   object
 1   Tm        5655 non-null   object
 2   FantPos   5655 non-null   object
 3   Age       5655 non-null   object
 4   G         5655 non-null   object
 5   GS        5655 non-null   object
 6   Cmp       5655 non-null   object
 7   Pass_Att  5655 non-null   object
 8   Pass_Yds  5655 non-null   object
 9   Pass_TD   5655 non-null   object
 10  Int       5655 non-null   object
 11  Rush_Att  5655 non-null   object
 12  Rush_Yds  5655 non-null   object
 13  Y/A       5655 non-null   object
 14  Rush_TD   5655 non-null   object
 15  Tgt       5655 non-null   object
 16  Rec       5655 non-null   object
 17  Rec_Yds   5655 non-null   object
 18  Y/R       5655 non-null   object
 19  Rec_TD    5655 non-null   object
 20  Fmb       5655 non-null   object
 21  FL        5655 non-

In [14]:
# converting columns to appropriate data types
df_nfl.replace('', 0, inplace=True) # this line is needed because empty strings cannot be converted to ints or floats
int_columns = ['Age', 'G', 'GS', 'PosRank', 'OvRank', 'Year']
df_nfl[int_columns] = df_nfl[int_columns].astype(int)

float_columns = ['Cmp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'Int', 'Rush_Att', 'Rush_Yds', 'Y/A', 'Rush_TD', 
                 'Tgt', 'Rec', 'Rec_Yds', 'Y/R', 'Rec_TD', 'Fmb', 'FL', 'Tot_TD', '2PM', '2PP','FantPt', 'PPR', 
                 'DKPt', 'FDPt', 'VBD']
df_nfl[float_columns] = df_nfl[float_columns].astype(float)

df_nfl['FantPos'][df_nfl['FantPos']=='FB'] == 'RB' # changing position of any fullbacks to runningbacks because in fantasy fbs are considered rbs 

df_nfl.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5655 entries, 0 to 5838
Data columns (total 33 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Player    5655 non-null   object 
 1   Tm        5655 non-null   object 
 2   FantPos   5655 non-null   object 
 3   Age       5655 non-null   int64  
 4   G         5655 non-null   int64  
 5   GS        5655 non-null   int64  
 6   Cmp       5655 non-null   float64
 7   Pass_Att  5655 non-null   float64
 8   Pass_Yds  5655 non-null   float64
 9   Pass_TD   5655 non-null   float64
 10  Int       5655 non-null   float64
 11  Rush_Att  5655 non-null   float64
 12  Rush_Yds  5655 non-null   float64
 13  Y/A       5655 non-null   float64
 14  Rush_TD   5655 non-null   float64
 15  Tgt       5655 non-null   float64
 16  Rec       5655 non-null   float64
 17  Rec_Yds   5655 non-null   float64
 18  Y/R       5655 non-null   float64
 19  Rec_TD    5655 non-null   float64
 20  Fmb       5655 non-null   float64
 

In [42]:
# players for the upcoming 2024 season
# I found a csv file online of someones fantasy ranks for the 2024 season
df_nfl_2024 = pd.read_csv('fantasyplayers2024.csv')
df_nfl_2024 = df_nfl_2024[['Player', 'Team', 'Pos']]
df_nfl_2024['Year'] = 2024

# changing column names to match the rest of the data
df_nfl_2024.rename(columns={'Team': 'Tm', 'Pos':'FantPos'}, inplace=True)

# the csv file I found doesn't have age so I'm going to get the age from my scraped data
df_nfl_2024 = pd.merge(df_nfl_2024, df_nfl[['Player', 'Age']][df_nfl['Year']==2023], how='left', on='Player')

# this line won't run until you run the part of the script that scrapes college data further down
df_nfl_2024 = pd.merge(df_nfl_2024, df_college_stats_adj[['Player', 'Age']][df_college_stats_adj['Year']==2024], how='left', on='Player')

# # fixing the columns
df_nfl_2024['Age'] = df_nfl_2024['Age_x'].fillna(df_nfl_2024['Age_y'])
df_nfl_2024.drop(['Age_x', 'Age_y'], axis=1, inplace=True)
df_nfl_2024['Age'] += 1
df_nfl_2024.dropna(inplace=True)

df_nfl_2024.head()

,Player,Tm,FantPos,Year,Age
0,CeeDee Lamb,DAL,WR,2024,25.0
1,Christian McCaffrey,SFO,RB,2024,28.0
2,Justin Jefferson,MIN,WR,2024,25.0
3,Tyreek Hill,MIA,WR,2024,30.0
4,Breece Hall,NYJ,RB,2024,23.0


In [43]:
# combining with the rest of the nfl data
df_nfl = pd.concat([df_nfl, df_nfl_2024]).reset_index(drop=True)

In [44]:
import sqlite3

In [45]:
# intilializing a connection to a sqlite database
conn = sqlite3.connect('fantasy_football_data.db')

In [46]:
# storing the dataframe in the database
df_nfl.to_sql('player_stats', conn, if_exists='replace', index=False)

5933

In [47]:
# quereying the whole dataframe to make sure everything worked properly
pd.read_sql('SELECT * FROM player_stats', conn)

,Player,Tm,FantPos,Age,G,GS,Cmp,Pass_Att,Pass_Yds,Pass_TD,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,Year
0,Antonio Brown,PIT,WR,27.0,16.0,16.0,0.0,0.0,0.0,0.0,...,2.0,0.0,252.0,388.2,393.2,320.2,125.0,1.0,1.0,2015
1,Devonta Freeman,ATL,RB,23.0,15.0,13.0,0.0,0.0,0.0,0.0,...,0.0,0.0,243.0,316.4,324.4,279.9,120.0,1.0,2.0,2015
2,Julio Jones,ATL,WR,26.0,16.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,239.0,375.1,379.1,307.1,111.0,2.0,3.0,2015
3,Adrian Peterson,MIN,RB,30.0,16.0,16.0,0.0,0.0,0.0,0.0,...,0.0,0.0,231.0,260.7,269.7,245.7,107.0,2.0,4.0,2015
4,Cam Newton,CAR,QB,26.0,16.0,16.0,296.0,495.0,3837.0,35.0,...,0.0,0.0,389.0,389.1,409.1,399.1,105.0,1.0,5.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5928,Kendre Miller,NOR,RB,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
5929,Keaton Mitchell,BAL,RB,22.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
5930,D'Onta Foreman,CLE,RB,28.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024
5931,Dylan Laube,LVR,RB,25.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024


# Data Scraping and Cleaning for College and Draft Data
I scraped college and draft data to make predictions for rookies

In [16]:
# loop to scrape college passing stats
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
dfs = []
for year in years:
    url = f'https://www.sports-reference.com/cfb/years/{year}-passing.html'
    df_year = scrape(url, 1)
    df_year['Year'] = year
    dfs.append(df_year)
df_pass = pd.concat(dfs, ignore_index=True)

In [17]:
df_pass

,Player,Team,Conf,G,Cmp,Att,Cmp%,Yds,TD,TD%,Int,Int%,Y/A,AY/A,Y/C,Y/G,Rate,Awards,OvRank,Year
0,Brandon Doughty*,Western Kentucky,CUSA,13,375,552,67.9,4830,49,8.9,10,1.8,8.8,9.71,12.9,371.5,167.1,,1,2014
1,Shane Carden*,East Carolina,American,13,392,617,63.5,4736,30,4.9,10,1.6,7.7,7.92,12.1,364.3,140.8,,2,2014
2,Marcus Mariota*,Oregon,Pac-12,15,304,445,68.3,4454,42,9.4,4,0.9,10.0,11.49,14.7,296.9,181.7,"H,Maxwell,AA",3,2014
3,Garrett Grayson*,Colorado State,MWC,13,270,420,64.3,4006,32,7.6,7,1.7,9.5,10.31,14.8,308.2,166.2,,4,2014
4,Jared Goff,California,Pac-12,12,316,509,62.1,3973,35,6.9,7,1.4,7.8,8.56,12.6,331.1,147.6,,5,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117,Jordan Nabors,Baylor,Big 12,9,1,1,100.0,-6,0,0.0,0,0.0,-6.0,-6.00,-6.0,-0.7,49.6,,537,2023
5118,Kyre Duplessis*,Coastal Carolina,Sun Belt,13,1,2,50.0,-7,0,0.0,0,0.0,-3.5,-3.50,-7.0,-0.5,20.6,,538,2023
5119,LeQuint Allen*,Syracuse,ACC,13,1,2,50.0,-9,0,0.0,0,0.0,-4.5,-4.50,-9.0,-0.7,12.2,,539,2023
5120,Dewayne Coleman,Army,Ind,3,1,1,100.0,-14,0,0.0,0,0.0,-14.0,-14.00,-14.0,-4.7,-17.6,,540,2023


In [18]:
# dropping null values
df_pass.dropna(inplace=True)

# adjusting column names
columns = df_pass.columns.tolist()
columns[5] = 'Pass_Att'
columns[6] = 'Cmp_Pct'
columns[7] = 'Pass_Yds'
columns[8] = 'Pass_TD'
columns[12] = 'Pass_Y/A'
columns[16] = 'PR'
df_pass.columns = columns

# dropping unneeded columns
df_pass.drop(['TD%', 'Int%', 'Y/C', 'Y/G', 'Awards', 'OvRank'], axis=1, inplace=True)

# deleting special characters in player names
df_pass['Player'] = df_pass['Player'].str.replace(r'[*]', '', regex=True)

# change column datatypes to appropriate types
df_pass.replace('', 0, inplace=True)
int_columns = ['G', 'Year']
df_pass[int_columns] = df_pass[int_columns].astype(int)
float_columns = ['Cmp', 'Pass_Att', 'Cmp_Pct', 'Pass_Yds', 'Pass_Y/A', 'AY/A', 'Pass_TD', 'Int', 'PR']
df_pass[float_columns] = df_pass[float_columns].astype(float)

# drop duplicates so only a player's most recent season is left
df_pass.drop_duplicates('Player', keep='last', inplace=True)

df_pass.head()

,Player,Team,Conf,G,Cmp,Pass_Att,Cmp_Pct,Pass_Yds,Pass_TD,Int,Pass_Y/A,AY/A,PR,Year
1,Shane Carden,East Carolina,American,13,392.0,617.0,63.5,4736.0,30.0,10.0,7.7,7.92,140.8,2014
2,Marcus Mariota,Oregon,Pac-12,15,304.0,445.0,68.3,4454.0,42.0,4.0,10.0,11.49,181.7,2014
3,Garrett Grayson,Colorado State,MWC,13,270.0,420.0,64.3,4006.0,32.0,7.0,9.5,10.31,166.2,2014
5,Jameis Winston,Florida State,ACC,13,305.0,467.0,65.3,3907.0,25.0,18.0,8.4,7.70,145.5,2014
6,Rakeem Cato,Marshall,CUSA,14,267.0,451.0,59.2,3903.0,40.0,13.0,8.7,9.13,155.4,2014


In [19]:
df_pass['Player'].duplicated().sum()

0

In [20]:
df_pass.query('Player == "Trevor Lawrence"')

,Player,Team,Conf,G,Cmp,Pass_Att,Cmp_Pct,Pass_Yds,Pass_TD,Int,Pass_Y/A,AY/A,PR,Year
3113,Trevor Lawrence,Clemson,ACC,10,231.0,334.0,69.2,3153.0,24.0,5.0,9.4,10.2,169.2,2020


In [21]:
# loop to scrape college rushing stats
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
dfs = []
for year in years:
    url = f'https://www.sports-reference.com/cfb/years/{year}-rushing.html'
    df_year = scrape(url, 6)
    df_year['Year'] = year
    dfs.append(df_year)
df_rush = pd.concat(dfs, ignore_index=True)

In [22]:
# dropping null values
df_rush.dropna(inplace=True)

# adjusting column names
columns = df_rush.columns.tolist()
columns[4] = 'Rush_Att'
columns[5] = 'Rush_Yds'
columns[6] = 'Rush_Y/A'
columns[7] = 'Rush_TD'
columns[9] = 'Rec_Yds'
columns[10] = 'Y/R'
columns[11] = 'Rec_TD'
columns[13] = 'Tot_Yds'
columns[14] = 'Y/P'
columns[15] = 'Tot_TD'
df_rush.columns = columns

# drop unneeded columns
df_rush.drop('OvRank', axis=1, inplace=True)

# deleting special characters in player names
df_rush['Player'] = df_rush['Player'].str.replace(r'[*]', '', regex=True)

# change column datatypes to appropriate types
df_rush.replace('', 0, inplace=True)
int_columns = ['G', 'Year']
df_rush[int_columns] = df_rush[int_columns].astype(int)
float_columns = ['Rush_Att', 'Rush_Yds', 'Rush_Y/A', 'Rush_TD', 'Rec', 'Rec_Yds', 'Y/R', 'Rec_TD', 'Plays', 'Tot_Yds', 'Y/P', 
                 'Tot_TD']
df_rush[float_columns] = df_rush[float_columns].astype(float)

# drop duplicates so only a player's most recent season is left
df_rush.drop_duplicates('Player', keep='last', inplace=True)

df_rush.head()

,Player,School,Conf,G,Rush_Att,Rush_Yds,Rush_Y/A,Rush_TD,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD,Year
5,Melvin Gordon,Wisconsin,Big Ten,14,343.0,2587.0,7.5,29.0,19.0,153.0,8.1,3.0,362.0,2740.0,7.6,32.0,2014
6,Tevin Coleman,Indiana,Big Ten,12,270.0,2036.0,7.5,15.0,25.0,141.0,5.6,0.0,295.0,2177.0,7.4,15.0,2014
7,Steward Butler,Marshall,CUSA,12,107.0,798.0,7.5,7.0,3.0,21.0,7.0,0.0,110.0,819.0,7.4,7.0,2014
8,Noah Copeland,Navy,Ind,13,129.0,952.0,7.4,5.0,3.0,65.0,21.7,1.0,132.0,1017.0,7.7,6.0,2014
9,Todd Gurley,Georgia,SEC,6,123.0,911.0,7.4,9.0,12.0,57.0,4.8,0.0,135.0,968.0,7.2,9.0,2014


In [23]:
# loop to scrape college receiving stats
years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
dfs = []
for year in years:
    url = f'https://www.sports-reference.com/cfb/years/{year}-receiving.html'
    df_year = scrape(url, 6)
    df_year['Year'] = year
    dfs.append(df_year)
df_rec = pd.concat(dfs, ignore_index=True)

In [24]:
# dropping null values
df_rec.dropna(inplace=True)

# adjusting column names
columns = df_rec.columns.tolist()
columns[5] = 'Rec_Yds'
columns[6] = 'Y/R'
columns[7] = 'Rec_TD'
columns[8] = 'Rush_Att'
columns[9] = 'Rush_Yds'
columns[10] = 'Rush_Y/A'
columns[11] = 'Rush_TD'
columns[13] = 'Tot_Yds'
columns[14] = 'Y/P'
columns[15] = 'Tot_TD'
df_rec.columns = columns

# drop unneeded columns
df_rec.drop('OvRank', axis=1, inplace=True)

# deleting special characters in player names
df_rec['Player'] = df_rec['Player'].str.replace(r'[*]', '', regex=True)

# change column datatypes to appropriate types
df_rec.replace('', 0, inplace=True) # this line is needed because empty strings cannot be converted to ints or floats
int_columns = ['G', 'Year']
df_rec[int_columns] = df_rec[int_columns].astype(int)
float_columns = ['Rush_Att', 'Rush_Yds', 'Rush_Y/A', 'Rush_TD', 'Rec', 'Rec_Yds', 'Y/R', 'Rec_TD', 'Plays', 'Tot_Yds', 'Y/P', 
                 'Tot_TD']
df_rec[float_columns] = df_rec[float_columns].astype(float)

# drop duplicates so only a player's most recent season is left
df_rec.drop_duplicates('Player', keep='last', inplace=True)

df_rec.head()

,Player,School,Conf,G,Rec,Rec_Yds,Y/R,Rec_TD,Rush_Att,Rush_Yds,Rush_Y/A,Rush_TD,Plays,Tot_Yds,Y/P,Tot_TD,Year
0,Devin Smith,Ohio State,Big Ten,15,33.0,931.0,28.2,12.0,0.0,0.0,0.0,0.0,33.0,931.0,28.2,12.0,2014
1,Phillip Dorsett,Miami (FL),ACC,13,36.0,871.0,24.2,10.0,2.0,-6.0,-3.0,0.0,38.0,865.0,22.8,10.0,2014
3,Sammie Coates,Auburn,SEC,12,34.0,741.0,21.8,4.0,0.0,0.0,0.0,0.0,34.0,741.0,21.8,4.0,2014
4,Ian Hamilton,UTEP,CUSA,13,27.0,579.0,21.4,2.0,0.0,0.0,0.0,0.0,27.0,579.0,21.4,2.0,2014
5,Breshad Perriman,UCF,American,13,50.0,1044.0,20.9,9.0,1.0,0.0,0.0,0.0,51.0,1044.0,20.5,9.0,2014


In [25]:
df_college = pd.concat([df_pass, df_rush, df_rec]).drop_duplicates('Player') # combine passing, rushing, and receiving stats
df_college.fillna(0, inplace=True) # fill in empty stats with 0s
df_college

,Player,Team,Conf,G,Cmp,Pass_Att,Cmp_Pct,Pass_Yds,Pass_TD,Int,...,Rush_Y/A,Rush_TD,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD
1,Shane Carden,East Carolina,American,13,392.0,617.0,63.5,4736.0,30.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Marcus Mariota,Oregon,Pac-12,15,304.0,445.0,68.3,4454.0,42.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Garrett Grayson,Colorado State,MWC,13,270.0,420.0,64.3,4006.0,32.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Jameis Winston,Florida State,ACC,13,305.0,467.0,65.3,3907.0,25.0,18.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Rakeem Cato,Marshall,CUSA,14,267.0,451.0,59.2,3903.0,40.0,13.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4821,John Jackson III,0,MWC,12,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,35.0,267.0,7.6,0.0,35.0,267.0,7.6,0.0
4827,Jacari Carter,0,Sun Belt,13,0.0,0.0,0.0,0.0,0.0,0.0,...,-8.0,0.0,38.0,276.0,7.3,0.0,39.0,268.0,6.9,0.0
4828,Jaylin Lucas,0,Big Ten,12,0.0,0.0,0.0,0.0,0.0,0.0,...,4.1,2.0,34.0,247.0,7.3,2.0,101.0,522.0,5.2,4.0
4829,E.J. Smith,0,Pac-12,12,0.0,0.0,0.0,0.0,0.0,0.0,...,4.1,1.0,33.0,242.0,7.3,0.0,86.0,460.0,5.3,1.0


In [26]:
df_college.duplicated('Player').sum()

0

In [27]:
df_college.query('Player == "Christian McCaffrey"')

,Player,Team,Conf,G,Cmp,Pass_Att,Cmp_Pct,Pass_Yds,Pass_TD,Int,...,Rush_Y/A,Rush_TD,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD
799,Christian McCaffrey,Stanford,Pac-12,14,2.0,3.0,66.7,39.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# loop to get nfl draft data
years = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
dfs = []
for year in years:
    url = f'https://www.pro-football-reference.com/years/{year}/draft.htm'
    df_year = scrape(url, 13)
    df_year['Year'] = year
    dfs.append(df_year)
df_draft = pd.concat(dfs, ignore_index=True)

In [29]:
df_draft = df_draft[['Year', 'Pick', 'Player', 'Tm', 'Pos', 'Age', 'College/Univ']].fillna(0) # select appropriate columns and fill na
df_draft.replace('', 0, inplace=True) # replace empty cells with 0
df_draft[['Pick', 'Age', 'Year']] = df_draft[['Pick', 'Age', 'Year']].astype(int) # adjust datatypes
df_draft

,Year,Pick,Player,Tm,Pos,Age,College/Univ
0,2015,1,Jameis Winston,TAM,QB,21,Florida St.
1,2015,2,Marcus Mariota,TEN,QB,21,Oregon
2,2015,3,Dante Fowler,JAX,OLB,21,Florida
3,2015,4,Amari Cooper,OAK,WR,21,Alabama
4,2015,5,Brandon Scherff,WAS,T,23,Iowa
...,...,...,...,...,...,...,...
2619,2024,253,Cornelius Johnson,LAC,WR,23,Michigan
2620,2024,254,KT Leveston,LAR,OL,24,Kansas St.
2621,2024,255,Kalen King,GNB,CB,21,Penn St.
2622,2024,256,Nick Gargiulo,DEN,OL,24,South Carolina


In [33]:
df_draft.to_sql('draft', conn, if_exists='replace', index=False)

2624

In [34]:
# merging draft information with college stats
df_college_stats = pd.merge(df_draft, df_college, on='Player', how='left') # omitting undrafted players because most undrafted players are not good

# cleaning the dataframe
df_college_stats = df_college_stats[df_college_stats['Pos'].isin(['QB', 'RB', 'WR', 'TE'])].reset_index(drop=True) # only selecting for fantasy position players
df_college_stats.rename(columns={'Year_x': 'Year', 'Year_y': 'College_Year'}, inplace=True)
df_college_stats.drop('College/Univ', axis=1, inplace=True)
df_college_stats

,Year,Pick,Player,Tm,Pos,Age,Team,Conf,G,Cmp,...,Rush_Y/A,Rush_TD,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD
0,2015,1,Jameis Winston,TAM,QB,21,Florida State,ACC,13.0,305.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015,2,Marcus Mariota,TEN,QB,21,Oregon,Pac-12,15.0,304.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015,4,Amari Cooper,OAK,WR,21,0,SEC,14.0,0.0,...,4.6,0.0,124.0,1727.0,13.9,16.0,129.0,1750.0,13.6,16.0
3,2015,7,Kevin White,CHI,WR,23,0,Big 12,13.0,0.0,...,0.0,0.0,109.0,1447.0,13.3,10.0,109.0,1447.0,13.3,10.0
4,2015,10,Todd Gurley,STL,RB,21,Georgia,SEC,6.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2024,235,Devaughn Vele,DEN,WR,26,Utah,Pac-12,10.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
787,2024,241,Tahj Washington,MIA,WR,23,0,Pac-12,13.0,0.0,...,10.0,0.0,59.0,1062.0,18.0,8.0,60.0,1072.0,17.9,8.0
788,2024,245,Michael Pratt,GNB,QB,22,Tulane,American,11.0,185.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
789,2024,246,Devin Culp,TAM,TE,24,0,Pac-12,13.0,0.0,...,2.0,0.0,29.0,266.0,9.2,1.0,30.0,268.0,8.9,1.0


In [35]:
# these null values are due to differences in names between college and draft data or missing data, so I need to manually go through and fix this
# the missing data is because the data I scraped does not have tight end data or fcs data
df_college_stats[df_college_stats.isnull().any(axis=1)].to_csv('college_stats_tochange.csv', index=False)
df_college.to_csv('college.csv', index=False)
df_college_stats[df_college_stats.isnull().any(axis=1)]

,Year,Pick,Player,Tm,Pos,Age,Team,Conf,G,Cmp,...,Rush_Y/A,Rush_TD,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD
5,2015,14,DeVante Parker,MIA,WR,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2015,40,Dorial Green-Beckham,TEN,WR,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2015,85,Tyler Kroft,CIN,TE,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2015,86,David Johnson,ARI,RB,23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2015,92,Jeff Heuerman,DEN,TE,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765,2024,166,Tyrone Tracy,NYG,RB,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,2024,167,Keilan Robinson,JAX,RB,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,2024,173,Isaiah Davis,NYJ,RB,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,2024,208,Dylan Laube,LVR,RB,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
# I adjusted the players that had name differences
# players with missing stats still remain
df_college_adj = pd.read_csv('college_adj.csv')

In [37]:
# remaking college_stats with college_adj
# merging draft information with college stats
df_college_stats = pd.merge(df_draft, df_college_adj, on='Player', how='left') # omitting undrafted players because most undrafted players are not good

# cleaning the dataframe
df_college_stats = df_college_stats[df_college_stats['Pos'].isin(['QB', 'RB', 'WR', 'TE'])].reset_index(drop=True) # only selecting for fantasy position players
df_college_stats.rename(columns={'Year_x': 'Year', 'Year_y': 'College_Year'}, inplace=True)
df_college_stats.drop(['OvRank', 'College/Univ'], axis=1, inplace=True)
df_college_stats

,Year,Pick,Player,Tm,Pos,Age,School,Conf,G,Cmp,...,College_Year,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD,Player_lower
0,2015,1,Jameis Winston,TAM,QB,21,Florida State,ACC,13.0,305.0,...,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,jameis winston
1,2015,2,Marcus Mariota,TEN,QB,21,Oregon,Pac-12,15.0,304.0,...,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,marcus mariota
2,2015,4,Amari Cooper,OAK,WR,21,Alabama,SEC,14.0,0.0,...,2014.0,124.0,1727.0,13.9,16.0,129.0,1750.0,13.6,16.0,amari cooper
3,2015,7,Kevin White,CHI,WR,23,West Virginia,Big 12,13.0,0.0,...,2014.0,109.0,1447.0,13.3,10.0,109.0,1447.0,13.3,10.0,kevin white
4,2015,10,Todd Gurley,STL,RB,21,Georgia,SEC,6.0,0.0,...,2014.0,12.0,57.0,4.8,0.0,135.0,968.0,7.2,9.0,todd gurley
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2024,235,Devaughn Vele,DEN,WR,26,Utah,Pac-12,10.0,0.0,...,2023.0,43.0,593.0,13.8,3.0,43.0,593.0,13.8,3.0,devaughn vele
787,2024,241,Tahj Washington,MIA,WR,23,USC,Pac-12,13.0,0.0,...,2023.0,59.0,1062.0,18.0,8.0,60.0,1072.0,17.9,8.0,tahj washington
788,2024,245,Michael Pratt,GNB,QB,22,Tulane,American,11.0,185.0,...,2023.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,michael pratt
789,2024,246,Devin Culp,TAM,TE,24,Washington,Pac-12,13.0,0.0,...,2022.0,29.0,266.0,9.2,1.0,30.0,268.0,8.9,1.0,devin culp


In [38]:
# all remaining players with null values are due to missing stats
# I'm going to manually fill in all the stats
df_college_stats.to_csv('college_stats.csv', index=False)
df_college_stats[df_college_stats.isnull().any(axis=1)]

,Year,Pick,Player,Tm,Pos,Age,School,Conf,G,Cmp,...,College_Year,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD,Player_lower
12,2015,40,Dorial Green-Beckham,TEN,WR,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,2015,85,Tyler Kroft,CIN,TE,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,2015,86,David Johnson,ARI,RB,23,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2015,92,Jeff Heuerman,DEN,TE,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,2015,117,Blake Bell,SFO,TE,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,2024,128,Ray Davis,BUF,RB,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
766,2024,167,Keilan Robinson,JAX,RB,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
769,2024,173,Isaiah Davis,NYJ,RB,22,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
780,2024,208,Dylan Laube,LVR,RB,24,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
# making college_stats_adj with filled in data
df_college_stats_adj = pd.read_csv('college_stats_adj.csv') # read in filled in dataframe
num_cols = ['Age', 'G', 'Cmp', 'Pass_Att', 'Cmp_Pct', 'Pass_Yds', 'Pass_Y/A', 'AY/A', 'Pass_TD', 'Int', 'PR', 'Rush_Att', 
            'Rush_Yds', 'Rush_Y/A', 'Rush_TD', 'Draft_Year', 'Rec', 'Rec_Yds', 'Y/R', 'Rec_TD', 'Plays',
            'Tot_Yds', 'Y/P', 'Tot_TD']     
df_college_stats_adj[num_cols] = df_college_stats_adj[num_cols].fillna(0) # fill in empty numerical stats because I didn't fill in 0s for all the stats that were 0
df_college_stats_adj.dropna(inplace=True) # drop players I couldn't find data for
df_college_stats_adj

,Year,Pick,Player,Tm,Pos,Age,School,Conf,G,Cmp,...,Rush_TD,Draft_Year,Rec,Rec_Yds,Y/R,Rec_TD,Plays,Tot_Yds,Y/P,Tot_TD
0,2015,1,Jameis Winston,TAM,QB,21,Florida State,ACC,13.0,305.0,...,3.0,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2015,2,Marcus Mariota,TEN,QB,21,Oregon,Pac-12,15.0,304.0,...,15.0,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2015,75,Garrett Grayson,NOR,QB,24,Colorado State,MWC,13.0,270.0,...,0.0,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2015,89,Sean Mannion,STL,QB,23,Oregon State,Pac-12,12.0,282.0,...,1.0,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2015,103,Bryce Petty,NYJ,QB,24,Baylor,Big 12,12.0,270.0,...,6.0,2014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2024,216,Ryan Flournoy,DAL,WR,24,SEMO,OVC,10.0,0.0,...,1.0,2023.0,57.0,839.0,14.7,6.0,63.0,888.0,14.1,7.0
787,2024,225,Brenden Rice,LAC,WR,22,USC,Pac-12,12.0,0.0,...,0.0,2023.0,45.0,791.0,17.6,12.0,45.0,791.0,17.6,12.0
788,2024,235,Devaughn Vele,DEN,WR,26,Utah,Pac-12,10.0,0.0,...,0.0,2023.0,43.0,593.0,13.8,3.0,43.0,593.0,13.8,3.0
789,2024,241,Tahj Washington,MIA,WR,23,USC,Pac-12,13.0,0.0,...,0.0,2023.0,59.0,1062.0,18.0,8.0,60.0,1072.0,17.9,8.0


In [40]:
df_college_stats_adj.isnull().sum()

Year          0
Pick          0
Player        0
Tm            0
Pos           0
Age           0
School        0
Conf          0
G             0
Cmp           0
Pass_Att      0
Cmp_Pct       0
Pass_Yds      0
Pass_Y/A      0
AY/A          0
Pass_TD       0
Int           0
PR            0
Rush_Att      0
Rush_Yds      0
Rush_Y/A      0
Rush_TD       0
Draft_Year    0
Rec           0
Rec_Yds       0
Y/R           0
Rec_TD        0
Plays         0
Tot_Yds       0
Y/P           0
Tot_TD        0
dtype: int64

In [41]:
df_college_stats_adj.to_sql('college_stats', conn, if_exists='replace', index=False)

752

In [43]:
conn.close()